### ST445 Managing and Visualizing Data
# Creating and managing databases
### Week 3 Lecture, MT 2017 - Kenneth Benoit, Christian Mueller

## Plan for today

- Relational and non-relational databases
- SQL and SQLite:
    - Creating tables
    - Querying data
    - Filtering
    - Grouping and aggregation
    - Joining tables
    - Adding indexes

## Relational data structures

-  invented by E. F. Codd at IBM in 1970
-  A relational database is a collection of data organized as a set
of formally defined tables
-  These tables can be accessed or reassembled in many different ways without having to reorganize the underlying tables that organize the data
-  RDBMS: a relational database management system. Examples include: MySQL, SQLite, PostgreSQL, Oracle. MS Access is a lite version of this too.
- The standard user and application programmer interface to a relational database is structured query language (SQL)

## Example

from: Database of Parties, Elections, and Governments (DPEG) relational database

```sql
SELECT c.countryName, c.countryAbbrev, p.* FROM party AS p
  LEFT JOIN country AS c
  ON p.countryID = c.countryID
```

## Database systems

#### Relational databases

- Mainly implementations and extensions of the SQL Standard ([ISO/IEC 9075:2016](https://www.iso.org/standard/63556.html))
- Transactions are always **ACID** (atomic, consistent, isolated, durable)
- Data needs to be defined

#### Non-relational databases

- Key-value storage types (e.g. Amazon DynamoDB) or document storage types (e.g. CouchDB, MongoDB)
- Sometime labelled as providing **ACID** transactions but often only _eventually consistent_

- FYI for clicking on the SQL standard link: The standard is open, i.e. anyone get it, but subject to a fee

## Basic SQL concepts

(Most of this will be familiar from last week's lecture and class on managing data)

- The basic unit is the **database**
- It might be stored on disk in a single file or a range of files managed by a server
- The database constists of **tables** which store actual data
- A table consist of at least one **column** whose name and data type need to be declared
- Data is stored in the **rows** of a table

## Basic relational structures

*  tables
   -  also known as “relations”
   -  tables define the forms of the data that are linked to other data through key relations

* keys: how tables are cross-referenced
   - **primary key**: an column in a table that uniquely identifies the remaining data in the table
   - **foreign key**: a field in a relational table that matches the primary key column of another table
   - **join operations** link tables in a structured query

## Normal forms 1

“Normalizing” a database means creating a proper set of relations

First normal form: No Repeating Elements or Groups of Elements

        countryname                   partyname   date per108 per110
    175     Austria           FPÖ Freedom Party 199010      3      0
    176     Austria        GA Green Alternative 199010      0      3
    177     Austria SPÖ Social Democratic Party 199010      5      0
    178     Austria          ÖVP People's Party 199010      8      0
    179     Austria           FPÖ Freedom Party 199410      1     11
    180     Austria            LF Liberal Forum 199410      0      0

Here, this is violated because of the wide format of per108 and per110.
To solve this, we have to move this to "long" format.


## Normal forms 2

Second normal form: No Partial Dependencies on a Concatenated Key

      countryname party   date category catcount
    1     Austria 42420 199010   per101        0
    2     Austria 42110 199010   per101        0
    3     Austria 42320 199010   per101        0
    4     Austria 42520 199010   per101        5
    5     Austria 42420 199410   per101        0
    6     Austria 42421 199410   per101        0

Here, the format is still violated, because party 42420 is repeated. To
solve this we need to create a party table and link to it using a
foreign key.

## Normal forms 3

Third normal form: No Dependencies on Non-Key Attributes. Every
non-prime attribute of data in a table must be dependent on a primary
key.

      countryname party   date category catcount
    1     Austria 42420 199010   per101        0
    2     Austria 42110 199010   per101        0
    3     Austria 42320 199010   per101        0
    4     Austria 42520 199010   per101        5
    5     Austria 42420 199410   per101        0
    6     Austria 42421 199410   per101        0

Here, this is violated because election data repeats across multiple
values of the category count table, when it should have its own table.

## Basic SQL syntax

- Defining data: `CREATE TABLE`
- Adding, changing, and deleting data: `INSERT`, `UPDATE`, `DELETE`
- Accessing data: `SELECT`
- Most functionality is part of the `SELECT` statement:
    - Filter: `SELECT ... WHERE`
    - Sort: `SELECT ... ORDER BY`
    - Aggregate: `SELECT ... GROUP BY`
    <!-- - Aggregate and filter: `SELECT ... GROUP BY ... HAVING` -->
    - Combining data: `SELECT ... JOIN`
- Adding constraints: `CREATE CONSTRAINT`
- Adding indexes: `CREATE INDEX`

## SQL Syntax caveats

- SQL syntax is **case-insensitive**
- `;` has to be added at the end of a line to terminate it (as in C-family languages, Javascript, ...)


## Setting up the SQLite command line


#### Installation via anaconda

```
conda install sqlite
```

#### Connecting to a database

```sh
sqlite st445-week03.db
```

```
SQLite version 3.20.1 2017-08-24 16:21:36
Enter ".help" for usage hints.
sqlite>
```

### Creating a table

```SQL
CREATE TABLE table_name (column_name column_type [, column_name column_type]) ;
```

Creating a table involves two things:

1. Giving the table a name constiting of alphanumeric characters and `_`
2. Giving each column a name and a **data type**

The SQL Standard defines several common data types and most SQL implementations provide additional ones:

| Type | Description                |
|:-----|:---------------------------|
| INT, BIGINT | Integer (4- and 8-byte). |
| FLOAT, DOUBLE | Single or double precision floating point number (4 or 8 bytes). |
| TEXT | String, stored using the database encoding (UTF-8, ...).|
| BLOB | Raw binary data. |
| BOOLEAN | True or false. |
| DATE, TIMESTAMP | Date and date-time. |

Compare data types available in [SQLite](https://www.sqlite.org/datatype3.html), [MySQL](https://dev.mysql.com/doc/refman/5.7/en/data-type-overview.html), and [PostgreSQL](https://www.postgresql.org/docs/current/static/datatype.html#DATATYPE-TABLE).

#### Example

```SQL
CREATE TABLE my_table (my_integer INT, my_float FLOAT, my_text TEXT) ;
```

## Adding data to a table

You probably will not do this by hand but rather via a script. This is the topic of the class.

```SQL
INSERT INTO table_name [(column_name [, column_name])] VALUES (value1 [, value2]);
```

#### Example

```SQL
INSERT INTO my_table VALUES (1, 1.3, 'abc') ;
INSERT INTO my_table (my_integer, my_float, my_text) VALUES (4, 4.3, 'def') ;
INSERT INTO my_table (my_integer, my_float) VALUES (8, 8.3) ;
```

[INSERT documentation](https://www.postgresql.org/docs/current/static/dml-insert.html)

## Changing or deleting data in a table

As above, you probably do not want to do this by hand.

```SQL
UPDATE table_name SET column_name = value [, column_name = value] WHERE ... ;
```

```SQL
DELETE FROM table_name WHERE ... ;
```

#### Example

```SQL
UPDATE my_table SET my_float = 1.5 WHERE my_integer = 1 ;
UPDATE my_table SET my_integer = my_integer * 2 WHERE my_text = 'def' ;
```


```SQL
DELETE FROM my_table WHERE my_integer = 8 ;
```

### Retrieving data

Whenever data should be retrieved, the statement starts with `SELECT`.

```SQL
SELECT column_name [, column_name] | * FROM table_name ;
```

- The most simple invocation selects all the columns with `*`

#### Example 1: Dummy table created above

```SQL
SELECT * FROM my_table ;
```

```
1|1.3|abc
4|4.3|def
```

#### Example 2: Actual data


```SQL
SELECT * FROM lecture_TBD ;
```

```
TBD
```

- This will return all the rows in the table which is not practical for tables with many rows
- To just have a glance at the first X rows of the data:
    ```SQL
    SELECT * FROM lecture_TBD LIMIT 20 ;
    ```


- This works the same as the `head(dat, n = 20)` function in R from last week
- **NB: There is no implicit ordering of rows in SQL**
- If the data is not explicitly ordered (will be explained shortly) there is no order whatsoever
- This is one of the main differences between SQL tables and tabular data in, e.g., an Excel sheet

## Filtering data

```SQL
SELECT ... WHERE condition ;
```
The `WHERE` clause is part and parcel of the power of SQL (`WHERE`, `JOIN`, and indexing)

You can test for many `condition`s:

- Equality: `WHERE column_name = value` (as used in the example above)
- Interval: `WHERE column_name BETWEEN lower_value AND higher_value`
- Set membership: `WHERE column_name IN (value1, value2, ...)`
- The set can be generated on the fly: `WHERE column_name IN (SELECT myvar FROM mytable)`
- Simple pattern match in strings: `WHERE string_column LIKE '_ab%'` 
    (`_` denotes _one_ character, `%` denotes zero or more characters, see also `GLOB` and `REGEX`)
- Missing values: `WHERE column_name IS NULL`

Individual `condition`s can be composed with the logic operators `AND`, `OR`, and `NOT`

* **Lab**: Working with a relational database manager
* **Next week**: Using data from the Internet